In [13]:
%%bash
docker run \
  --rm \
  --name omnisci_test \
  --ipc=host \
  -d --runtime=nvidia \
  -v $(readlink -f ..)/data:/data \
  -v $(readlink -f ..)/scripts:/src \
  -p 6273-6280:6273-6280 \
  omnisci/core-os-cuda:v5.2.1

c1513651a77dc98ea81cd4b2ba6eebc5abb88fa9ba3be267db65e8930fd54d93


In [15]:
%%bash
docker exec omnisci_test /bin/bash -c \
    "cat /src/load_dataset.sql | /omnisci/bin/omnisql -p 'HyperInteractive' -t"

User admin connected to database omnisci
Result
Loaded: 1067371 recs, Rejected: 0 recs in 0.308000 secs
1 rows returned.
Execution time: 391 ms, Total time: 391 ms
Result
Loaded: 5305 recs, Rejected: 0 recs in 0.027000 secs
1 rows returned.
Execution time: 33 ms, Total time: 33 ms
User admin disconnected from database omnisci


In [16]:
%%bash
docker exec omnisci_test /bin/bash -c \
    "echo 'SELECT * FROM online_retail LIMIT 10;' | /omnisci/bin/omnisql -p 'HyperInteractive' -t"

User admin connected to database omnisci
Invoice|StockCode|Description|Quantity|InvoiceDate|Price|CustomerID|Country
490353|21034|REX CASH+CARRY JUMBO SHOPPER|3|2009-12-04 16:37:00|0.95|15514|United Kingdom
490355|84879|ASSORTED COLOUR BIRD ORNAMENT|48|2009-12-04 16:39:00|1.69|13093|United Kingdom
490355|22189|CREAM HEART CARD HOLDER|36|2009-12-04 16:39:00|3.39|13093|United Kingdom
490355|21490|SET OF THREE 50'S GIFT WRAPS|107|2009-12-04 16:39:00|1.65|13093|United Kingdom
490355|21491|SET OF THREE VINTAGE GIFT WRAPS|96|2009-12-04 16:39:00|1.65|13093|United Kingdom
490354|21489|NULL|-23|2009-12-04 16:39:00|0|NULL|United Kingdom
490356|35957|SMALLFOLKART BAUBLE CHRISTMAS DEC|1|2009-12-04 16:45:00|0.42|17884|United Kingdom
490938|84678|CLASSICAL ROSE SMALL VASE|1|2009-12-08 13:59:00|2.55|14605|United Kingdom
491646|84813|SET OF 4 DIAMOND NAPKIN RINGS|8|2009-12-11 17:00:00|3.95|14031|United Kingdom
490356|22086|PAPER CHAIN KIT 50'S CHRISTMAS|12|2009-12-04 16:45:00|2.95|17884|United Kingdom

In [6]:
from pymapd import connect
from time import time
import pandas as pd
import numpy as np
import json

In [7]:
with open('test_result.json') as json_file:
    test_result = json.load(json_file)

In [8]:
con = connect(user="admin", password="HyperInteractive", host="localhost", dbname="omnisci")

In [9]:
test_result["timings"]["omnisci"] = {}

for label, query in test_result["queries"].items():
    print(f"Executing {label} -> {query}")
    test_result["timings"]["omnisci"][label] = []
    for i in range(0, 10):
        start = time()
        pd.read_sql_query(query, con=con)  # <-un po' più veloce ma usa ipc (arrow)
        # df = con.select_ipc_gpu(query)  # <-analogo
        # result = list(con.execute(query))  #NO IPC
        end = time()

        total = end-start
        test_result["timings"]["omnisci"][label].append(total)

with open('test_result.json', 'w') as outfile:
    json.dump(test_result, outfile)

Executing Q1 -> SELECT Invoice, SUM(Price*Quantity) as invoice_sum FROM online_retail GROUP BY Invoice ORDER BY invoice_sum DESC
Executing Q2 -> SELECT Country, SUM(Price*Quantity) as country_sum FROM online_retail GROUP BY Country ORDER BY country_sum DESC
Executing Q3 -> SELECT StockCode, COUNT(*) as n_items FROM online_retail GROUP BY StockCode
Executing Q4 -> SELECT CustomerID, SUM(Price*Quantity) as customer_sum FROM online_retail GROUP BY CustomerID ORDER BY customer_sum DESC
Executing Q5 -> SELECT InvoiceDate, SUM(Price*Quantity) as price_sum FROM online_retail GROUP BY InvoiceDate  ORDER BY price_sum
Executing Q6 -> SELECT CustomerID, Price*Quantity as total_price FROM online_retail WHERE Quantity > 20 ORDER BY total_price DESC
Executing Q7 -> SELECT InvoiceDate, AVG(Price*Quantity) as avg_total FROM online_retail GROUP BY InvoiceDate


## Join tests

In [10]:
query = "SELECT Invoice, stock_codes.Price FROM online_retail JOIN stock_codes ON online_retail.StockCode = stock_codes.StockCode"

start = time()
pd.read_sql_query(query, con=con)
end = time()

total = end-start
print(total)

1.6789634227752686


In [11]:
query = "SELECT Invoice, Price FROM online_retail"

start = time()
pd.read_sql_query(query, con=con)
end = time()

total = end-start
print(total)

1.4988090991973877


In [17]:
%%bash
docker stop omnisci_test

omnisci_test
